# COGS 108 - Data Checkpoint

# Names

- Anna Wang
- Chloe Salem
- Kristy Liou
- Maxtierney Arias
- Zeven Vidmar Barker

<a id='research_question'></a>
# Research Question

Can we predict which state in the USA will be covid-free first based on current hospital records, state regulations, and population?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: **Population, Population Changes, and Estimates**
- Link to the dataset: https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/national/totals/nst-est2020.csv
- Number of observations: The US Census dataset from 2010 includes populations of the country, states, and regions as well as estimates of each for every year leading up to 2020. This dataset is the best available given that the 2020 census is still being processed.


- Dataset Name: **US State Vaccinations**
- Link to the dataset: https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/us_state_vaccinations.csv
- It has 2104 observations, which include data on how many people from each state has been vaccinated starting from January 12, 2021.  These observations are broken down by day, which will allow us to analyze the rate at which vaccines are being received and distributed in each state. The dataset features the total number of vaccinations a state has each day with the total number of vaccinations distributed per day.


- Dataset Name: **COVID Tracking**
- Link to the dataset: https://covidtracking.com/data
- It has 2,006 observations, that shows us the number of COVID cases for each state in the US since January 12, 2020, along with patient hospitalization data by state, data on deaths, and COVID testing information. We will be utilizing hospitalization data and testing information.





# Setup

In [60]:
#importing needed libraries
import pandas as pd
import seaborn as sns
import numpy as np

# reading data sets
population = pd.read_csv("https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/national/totals/nst-est2020.csv")
vaccinations = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')
# includes hospitalization and covid cases over time
case_tracking = pd.read_csv('https://covidtracking.com/data/download/all-states-history.csv')

# Data Cleaning

The three datasets we used were already in a tidy format, with variables representing every distinct measurement made by the sources in the columns, and separate observations in the rows. Since we did remove observations unrelated to states in the Census, and we just wanted the states and their 2020 population estimates, we reset the index for `population`. For the other two datasets, we just removed the variables that were unnecessary for our analysis.

To answer our question, we will utilize data on hospital capacity by state, vaccinations in each state, and COVID cases by state, combined with population data sourced from the US Census. 


In [61]:
# Removing the regions population
population = population[5::]
# Remove columns with years outside 2020
population = population.drop(population.columns[7:-1], 1)
# Remove unnecessary variables related to region and identifiers
population = population.drop(['STATE','SUMLEV', 'DIVISION', 'REGION','CENSUS2010POP','ESTIMATESBASE2010'], 1)
# Set index to be the name of the state, rather than an arbitrary number
population.set_index(['NAME'], inplace=True)
population.head()

,POPESTIMATE2020
NAME,
Alabama,4921532
Alaska,731158
Arizona,7421401
Arkansas,3030522
California,39368078


In [62]:
#drop columns unrelated to research question
vaccinations = vaccinations.drop(['daily_vaccinations_per_million', 'share_doses_used', 'daily_vaccinations_per_million', 'share_doses_used'], 1)
vaccinations.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.52,7.73,5906.0,5906.0
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.27,2.05,13488.0,1.77,9.07,8267.0,7478.0
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0


In [63]:
#dropping more unrelated columns to research question - including deaths, antibody tests, negative results, and positive results related to type of test
case_tracking = case_tracking.drop(['dataQualityGrade','deathProbable', 'death', 'deathConfirmed', 'deathIncrease', 'negative',
       'negativeIncrease', 'negativeTestsAntibody',
       'negativeTestsPeopleAntibody', 'negativeTestsViral', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease','positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'inIcuCumulative','inIcuCurrently','onVentilatorCumulative', 'onVentilatorCurrently'], 1)
case_tracking.head()

,date,state,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,positive,recovered,totalTestEncountersViral,totalTestEncountersViralIncrease,totalTestResults,totalTestResultsIncrease
0,2021-02-12,AK,1230.0,1230.0,35.0,3,54282.0,NaN,NaN,0,1584548.0,7192
1,2021-02-12,AL,44148.0,44148.0,1267.0,242,478667.0,264621.0,NaN,0,2218143.0,6772
2,2021-02-12,AR,14278.0,14278.0,712.0,23,311608.0,293796.0,NaN,0,2569944.0,6669
3,2021-02-12,AS,NaN,NaN,NaN,0,0.0,NaN,NaN,0,2140.0,0
4,2021-02-12,AZ,55413.0,55413.0,2396.0,141,793532.0,110642.0,NaN,0,7140917.0,35221


# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/13  | 4 PM  | Import & Wrangle Data; EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis| Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |